In [ ]:
#test notebook for development of Xtal implementation in SRW
#in the end, the spectra are shown, which indicate the correctness of the calculations
#by I. Petrov, L. Samoylova
#X-ray Optics, European XFEL


from __future__ import print_function #Python 2.7 compatibility
from __future__ import absolute_import
from __future__ import division
from __future__ import unicode_literals

import sys
sys.path.insert(0, '../../..')

import numpy as np
import pylab as plt
import os
import matplotlib
import glob

from wpg.srwlib import srwl
from wpg.srwlib import *
from wpg import Beamline, optical_elements
from wpg.optical_elements import Drift, Use_PP
from wpg.wavefront import Wavefront
from wpg.generators import build_gauss_wavefront

from wpg.useful_code.wfrutils import show_slices_hsv
from wpg.wpg_uti_exfl import calculate_theta_fwhm_cdr_s1
from wpg.wpg_uti_wf import calculate_fwhm, show_profile, time_en_int, plot_t_wf, integral_intensity

In [ ]:
#define crystal parameters, can be found here http://x-server.gmca.aps.anl.gov/
beam_ph_en = 8
Laue = 1 #When Laue == 1 is Laue geometry if Laue == 0 Bragg geometry
Transmission = 0 # Transmission == 0 for diffraction, Transmision == 1 for Transmitted

if Laue == 0:
    if Transmission == 0:
        uc = 1
    else:
        uc = 2
elif Laue == 1:
    if Transmission == 0:
        uc = 3
    else:
        uc = 4

print('CASE:',uc, '  (1- BraggR, 2- BraggF, 3- LaueR, 4- LaueT)')

#Si111
dSp = 3.1355 #lattice spacing Angstrom


#susceptibilities, vary with photon energy, for your crystal,
#specify values for 12.4 keV (that are before division)

psi0r = -0.63324e-05/(1+(beam_ph_en-12.4)/12.4)**2 
psi0i =  0.63400e-07/(1+(beam_ph_en-12.4)/12.4)**2
psihr = -0.33401e-05/(1+(beam_ph_en-12.4)/12.4)**2
psihi =  0.44215e-07/(1+(beam_ph_en-12.4)/12.4)**2
psihbr = psihr; psihbi = psihi 


#thickness
thickCryst = 0.1e-03 

#asymmetry angle, 0 for Bragg, np.pi/2 for Laue
angAsCryst = 0

In [ ]:
#1 crystals in (n) orientation
opCr1 = SRWLOptCryst(_d_sp=dSp, _psi0r=psi0r,_psi0i=psi0i, \
_psi_hr=psihr, _psi_hi=psihi,_psi_hbr=psihbr, _psi_hbi=psihbi,\
                     _tc=thickCryst, _ang_as=angAsCryst,_uc =uc)

BraggAngle1 = opCr1.Bragg_angle(en_kev=beam_ph_en)

opCr1.find_orient_and_rotate_faceUpDown(en_kev=beam_ph_en,UpDown=0)

prPar1 =    [0, 0, 1.0, 0, 0, 1.0,  1.0,    1.0,    1.0, 0, 0, 0]
#??
optBLCryst2 = SRWLOptC([opCr1], [prPar1])

print(opCr1.nvx,opCr1.nvy,opCr1.nvz)
print(opCr1.tvx,opCr1.tvy)
print(1/tan(BraggAngle1))

In [ ]:
print('nvx,nvy,nvz',opCr1.nvx,opCr1.nvy,opCr1.nvz)
print('tvx,tvy',opCr1.tvx,opCr1.tvy)

In [ ]:
#Gaussian beam parameters, this can be used for demonstration of rocking curve width

beam_ph_en = 8      #Gaussian beam mean photon energy
pulseInt = 1e-3        #pulse intensity
coh_time = 0.05e-15     #coherence time, s
theta_fwhm = 2.0e-6    #2.0e-6  #angular divergence, rad
num_points = 200       #400 #number of points in in transverse direction
nslices = 100          #100 #number of points in time
sig_num = 5.5         #5.5          #number of sigmas in transverse direction
src_oe1 = 840.8        #distance from source, m

wlambda = 12.4*1e-10/beam_ph_en #Wavelenght
omega0 = 2*np.pi*3*1e8/wlambda  #Width of the Gaussian?
w0 = wlambda/(np.pi*theta_fwhm)
w0_a = w0/(2*np.sqrt(np.log(2)))
pulseRange = 20.0 # 20.

zR = (np.pi*w0_a**2)/wlambda 
range_xy = w0_a * np.sqrt(1+(src_oe1/zR)**2)/np.sqrt(2)*sig_num

#Build gaussian wavefront
wfr = build_gauss_wavefront(num_points, num_points, nslices, \
beam_ph_en,-range_xy/2,range_xy/2, -range_xy/2, range_xy/2,
coh_time/np.sqrt(2), w0_a, w0_a, src_oe1, pulseEn=pulseInt, pulseRange = pulseRange)

wf = Wavefront(wfr)

In [ ]:
#show beam profile and calculate beamsize in m
show_profile(wf)

In [ ]:
#plot time plots of total power and in the center of beam
integral_intensity(wf)

In [ ]:
#switch to frequency representation, 
srwl.ResizeElecField(wf._srwl_wf, 't', [0.0,10.0,0.1])
srwl.SetRepresElecField(wf._srwl_wf, 'f')
srwl.ResizeElecField(wf._srwl_wf, 't', [0.0,0.3,3.0])

In [ ]:
#plot energy plots of total power and in the center of beam
integral_intensity(wf)
srwl.PropagElecField(wf._srwl_wf,optBLCryst2)

In [ ]:
integral_intensity(wf)

In [ ]:
srwl.SetRepresElecField(wf._srwl_wf, 't')

In [ ]:
integral_intensity(wf)